In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-experimental
!pip install pypdf
!pip install openai
!pip install tenacity
!pip install faiss-cpu
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai
from langchain_openai import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.2 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()
#OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain import LLMChain
from langchain.schema import HumanMessage
from langchain.callbacks import get_openai_callback
from langchain.prompts import PromptTemplate

from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from fastapi import FastAPI


In [ ]:
Doc = "/content/drive/MyDrive/RAG_Task/Basic Network Troubleshooting - 2015.pdf"



In [ ]:
#---- Indexing---
# loading the PDF

loader = PyPDFLoader(Doc)
pages = loader.load()

In [ ]:
#Spliting the doc into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
chunks = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
db_FAISS = FAISS.from_documents(chunks,embeddings)

In [ ]:
# generating answer based on user query and retriving context information

#context_text = "\n\n".join([doc.page_content for doc in docs_faiss])



In [ ]:
# Prompt Template

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the context above: {question}.
Dont give information not mentioned in the CONTEXT INFORMATION."""

In [ ]:
#loading retriver contect and user query in the prompt template

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
#prompt = prompt_template.format_messages(context=context_text, question=query)

In [ ]:
# creating the RetrievalQA Chain
creating the RetrievalQA Chain

# Use prompt_template instead of PROMPT_TEMPLATE
chain_type_kwargs = {"prompt": prompt_template}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db_FAISS.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [ ]:
# Calling LLM to generate the answer based on the given context and query
#model = ChatOpenAI()
#message_content  = prompt[0].content
#response_text = model.predict(message_content)

In [ ]:
#print(response_text)

The major networking troubleshooting methods mentioned in the context are systematically searching each part of your network for problems, creating a network map, and using a network troubleshooting flowchart.


In [ ]:
query = input("Please enter your query: ")

result = qa({"query": query})
print("ANswer:\n", result['result'])

In [ ]:
# Create the FastAPI app

#app = FastAPI()

In [ ]:
#@app.get("/answer")
#async def get_answer(question: str):
 # """
  #Endpoint to get an answer to a question based on the document.
  #"""
  #result = qa({"query": question})
  #return {"answer": result['result']}

In [ ]:
query = input("Please enter your query: ")